In [ ]:
!pip install timm
!pip install pytorchvideo
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 3.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 961.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 KB 250.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188714 sha256=f775c51ae52c45d0e6b15305296d5059f0660f624138c975132e23ff472b8c24
  Stored in directory: /root/.c

In [ ]:
!git clone https://github.com/microsoft/Cream.git

Cloning into 'Cream'...
remote: Enumerating objects: 1844, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 1844 (delta 85), reused 73 (delta 49), pack-reused 1680
Receiving objects: 100% (1844/1844), 5.43 MiB | 5.67 MiB/s, done.
Resolving deltas: 100% (638/638), done.


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import csv
from torchvision.io import read_video
import random
from torch.utils.data import (Dataset, DataLoader)
from torchvision import transforms
from tqdm import tqdm
import sys
import time


# Kinetics Data

In [ ]:
!wget "https://kaistackr-my.sharepoint.com/:u:/g/personal/kietngt00_kaist_ac_kr/EfmUXPO9YsxJjU2XoZu5OToBeGmVDibD5UOghRkgO64OoQ?e=UCg5lM&download=1" \
-O Kinetics.zip

!wget "https://kaistackr-my.sharepoint.com/:x:/g/personal/kietngt00_kaist_ac_kr/ET_X7-lNmctPl9WsEmt37FEBslwbTfAVULh3TIJ5Pe5tLQ?e=wqgWW0&download=1" \
-O train_split.csv

!wget "https://kaistackr-my.sharepoint.com/:x:/g/personal/kietngt00_kaist_ac_kr/Ee36jEEKqSFEgKNJ39_LDDgBhnzLjaSrwlnlggDHa4kwQA?e=AyAASc&download=1" \
-O val_split.csv

--2023-03-16 23:48:06--  https://kaistackr-my.sharepoint.com/:u:/g/personal/kietngt00_kaist_ac_kr/EfmUXPO9YsxJjU2XoZu5OToBeGmVDibD5UOghRkgO64OoQ?e=UCg5lM&download=1
Resolving kaistackr-my.sharepoint.com (kaistackr-my.sharepoint.com)... 13.107.136.8, 13.107.138.8, 2620:1ec:8f8::8, ...
Connecting to kaistackr-my.sharepoint.com (kaistackr-my.sharepoint.com)|13.107.136.8|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/kietngt00_kaist_ac_kr/Documents/Kinetics-400/Kinetics-400.zip?ga=1 [following]
--2023-03-16 23:48:07--  https://kaistackr-my.sharepoint.com/personal/kietngt00_kaist_ac_kr/Documents/Kinetics-400/Kinetics-400.zip?ga=1
Reusing existing connection to kaistackr-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 20406211215 (19G) [application/x-zip-compressed]
Saving to: ‘Kinetics.zip’

Kinetics.zip        100%[===================>]  19.00G  76.9MB/s    in 6m 32s  

2023-03-16 23:54:39 (49.7 MB/s) - ‘Kinetics.zip’ 

In [ ]:
!unzip Kinetics.zip

Streaming output truncated to the last 5000 lines.
  inflating: Kinetics-400/ZPYOYq2bVtE_000003_000013.mp4  
  inflating: Kinetics-400/ZpZ5lPqKv_8_000296_000306.mp4  
  inflating: Kinetics-400/ZPZ9_khYyJ4_000001_000011.mp4  
  inflating: Kinetics-400/ZpZDeEyZsvw_000001_000011.mp4  
  inflating: Kinetics-400/ZPzOxiegBvM_000013_000023.mp4  
  inflating: Kinetics-400/ZpzPgLbu0YA_000004_000014.mp4  
  inflating: Kinetics-400/ZpZrCcdGerk_000000_000010.mp4  
  inflating: Kinetics-400/ZPZyrxW1h6s_000028_000038.mp4  
  inflating: Kinetics-400/Zp_WLkJyPmw_000057_000067.mp4  
  inflating: Kinetics-400/ZQ-dq16xkDI_000003_000013.mp4  
  inflating: Kinetics-400/ZQ-S5HzkSwI_000003_000013.mp4  
  inflating: Kinetics-400/ZQ-zFOca8RM_000021_000031.mp4  
  inflating: Kinetics-400/Zq0f6LjYW_g_000052_000062.mp4  
  inflating: Kinetics-400/Zq0ruOa_UYg_000279_000289.mp4  
  inflating: Kinetics-400/Zq1vGF1gnYA_000000_000010.mp4  
  inflating: Kinetics-400/Zq1_en3_ByM_000002_000012.mp4  
  inflating: Kinetics

In [ ]:
!rm -rf Kinetics.zip

## Data

In [ ]:
class KineticsDataset(Dataset):
    def __init__(self, data, root, video_transforms=None, frame_transforms=None):
        self.root = root
        self.data = data # The df load from train/val split
        self.video_transforms = video_transforms
        self.frame_transforms = frame_transforms
        self.num_samples = data.shape[0]
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        file_name, label = row[0], row[1]
        vid = read_video(os.path.join(self.root, file_name), output_format='TCHW', pts_unit='sec')[0]
        
        if self.frame_transforms is not None:
            vid = torch.stack(self.frame_transforms(frame) for frame in vid)
        vid = vid.transpose(0,1) # CTHW
        if self.video_transforms is not None:
            vid = self.video_transforms(vid)
            
        return vid, torch.tensor(label)


In [ ]:
from pytorchvideo.transforms import ( # Input(C,T,H,W)
    ApplyTransformToKey,
    RandomShortSideScale,
    ShortSideScale,
    UniformTemporalSubsample
)
from torchvision.transforms import (
    Compose,
    Lambda,
)
from torchvision.transforms._transforms_video import ( # Input: (C,T,H,W)
    CenterCropVideo,
    NormalizeVideo,
    RandomCropVideo,
    RandomHorizontalFlipVideo
)

train = pd.read_csv('./train_split.csv')
val = pd.read_csv('./val_split.csv')

random_crop_size = 256
img_size = 224

video_train_transforms = Compose([
    UniformTemporalSubsample(16),
    Lambda(lambda x: x / 255.0),
    # Lambda(lambda x: torch.transpose(x, 0, 1)), # (TCHW) -> (CTHW)
    NormalizeVideo((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
    RandomShortSideScale(min_size=random_crop_size, max_size=320),
    RandomCropVideo(img_size),
    RandomHorizontalFlipVideo(p=0.5),
])


video_val_transforms = Compose([
    UniformTemporalSubsample(16),
    Lambda(lambda x: x / 255.0),
    # Lambda(lambda x: torch.transpose(x, 0, 1)), # (TCHW) -> (CTHW)
    NormalizeVideo((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
    ShortSideScale(random_crop_size),
    CenterCropVideo(img_size)
])

train_dataset = KineticsDataset(train, './Kinetics-400', video_train_transforms)
val_dataset = KineticsDataset(val, './Kinetics-400', video_val_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=32)

/usr/local/lib/python3.9/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


# MRSDailyActivity3D

In [ ]:
!curl 'https://australiasoutheast1-mediap.svc.ms/transform/zip?cs=fFNQTw' \
  -H 'authority: australiasoutheast1-mediap.svc.ms' \
  -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7' \
  -H 'accept-language: en,vi;q=0.9,en-US;q=0.8' \
  -H 'cache-control: max-age=0' \
  -H 'content-type: application/x-www-form-urlencoded' \
  -H 'origin: https://uowmailedu-my.sharepoint.com' \
  -H 'sec-ch-ua: "Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Windows"' \
  -H 'sec-fetch-dest: iframe' \
  -H 'sec-fetch-mode: navigate' \
  -H 'sec-fetch-site: cross-site' \
  -H 'upgrade-insecure-requests: 1' \
  -H 'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36' \
  --data-raw 'zipFileName=MSRDailyActivity3D.zip&guid=501b8d4a-bbd4-4789-9fbb-639e9f4dd48f&provider=spo&files=%7B%22items%22%3A%5B%7B%22name%22%3A%22MSRDailyActivity3D%22%2C%22size%22%3A0%2C%22docId%22%3A%22https%3A%2F%2Fuowmailedu-my.sharepoint.com%3A443%2F_api%2Fv2.0%2Fdrives%2Fb%21uwGYadFtyU6OVnQDk_ZEhhKLwJYs1VZKihwaHlyHmhObZeBosGs8RYIuICJUSedv%2Fitems%2F01AORZ6DZLQ4MUZYOG5ZFJ2L2HYMRUBAZD%3Fversion%3DPublished%26access_token%3DeyJ0eXAiOiJKV1QiLCJhbGciOiJub25lIn0.eyJhdWQiOiIwMDAwMDAwMy0wMDAwLTBmZjEtY2UwMC0wMDAwMDAwMDAwMDAvdW93bWFpbGVkdS1teS5zaGFyZXBvaW50LmNvbUBhYWI0ODk3ZC01Zjk5LTQ0MzktYjQ0Mi1jMjA0YzY1ODc1YjUiLCJpc3MiOiIwMDAwMDAwMy0wMDAwLTBmZjEtY2UwMC0wMDAwMDAwMDAwMDAiLCJuYmYiOiIxNjc5MTQwODAwIiwiZXhwIjoiMTY3OTE2MjQwMCIsImVuZHBvaW50dXJsIjoiRVZxV0o2YUd2OHlhTU5MSTlybHlkNndUemVELy9rZW9aS0x6aVZDbzFxbz0iLCJlbmRwb2ludHVybExlbmd0aCI6IjEyMCIsImlzbG9vcGJhY2siOiJUcnVlIiwidmVyIjoiaGFzaGVkcHJvb2Z0b2tlbiIsInNpdGVpZCI6Ik5qazVPREF4WW1JdE5tUmtNUzAwWldNNUxUaGxOVFl0TnpRd016a3paalkwTkRnMiIsIm5hbWVpZCI6IjAjLmZ8bWVtYmVyc2hpcHx1cm4lM2FzcG8lM2Fhbm9uIzZlYjlkMTJmNjAwNmY0MjQ4NDYxMjlmODIyOGUxZDg4MGNjOTM0YzM0YzdlMTdhMjU5YmUyZWJhMWI4NjA3OTMiLCJuaWkiOiJtaWNyb3NvZnQuc2hhcmVwb2ludCIsImlzdXNlciI6InRydWUiLCJjYWNoZWtleSI6IjBoLmZ8bWVtYmVyc2hpcHx1cm4lM2FzcG8lM2Fhbm9uIzZlYjlkMTJmNjAwNmY0MjQ4NDYxMjlmODIyOGUxZDg4MGNjOTM0YzM0YzdlMTdhMjU5YmUyZWJhMWI4NjA3OTMiLCJzaGFyaW5naWQiOiJSRFZpd3lsL1prR2xRWkQzVk95NFpRIiwidHQiOiIwIiwidXNlUGVyc2lzdGVudENvb2tpZSI6IjIiLCJpcGFkZHIiOiIxMTAuNzYuMTA4LjIyOSJ9.dlFIWHloUHQwcU1ueEh3eWxlTGZoZW9zazkvWkxlejhRVFpxSi9aSXcwOD0%22%2C%22isFolder%22%3Atrue%7D%5D%7D&oAuthToken=' \
  --compressed --output MSRDailyActivity3D.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3398M    0 3398M  100  1564  12.5M      5  0:05:12  0:04:30  0:00:42 12.5M


In [ ]:
!unzip MSRDailyActivity3D.zip

Archive:  MSRDailyActivity3D.zip
 extracting: MSRDailyActivity3D/MSRDailyAct3D_pack1.zip  
 extracting: MSRDailyActivity3D/MSRDailyAct3D_pack2.zip  
 extracting: MSRDailyActivity3D/MSRDailyAct3D_pack3.zip  
 extracting: MSRDailyActivity3D/MSRDailyAct3D_pack4.zip  
 extracting: MSRDailyActivity3D/MSRDailyAct3D_pack5.zip  
 extracting: MSRDailyActivity3D/MSRDailyAct3D_pack6.zip  
 extracting: MSRDailyActivity3D/MSRDailyAct3D_pack7.zip  
 extracting: MSRDailyActivity3D/MSRDailyAct3D_pack8.zip  
 extracting: MSRDailyActivity3D/MSRDailyAct3DSampleCode.zip  


In [ ]:
!rm MSRDailyActivity3D/MSRDailyAct3DSampleCode.zip

In [ ]:
!mkdir data

In [ ]:
!unzip MSRDailyActivity3D/MSRDailyAct3D_pack1.zip -d data
!find . -name "*.bin" -type f -delete
!find . -name "*.txt" -type f -delete
!rm MSRDailyActivity3D/MSRDailyAct3D_pack1.zip

!unzip MSRDailyActivity3D/MSRDailyAct3D_pack2.zip -d data
!find . -name "*.bin" -type f -delete
!find . -name "*.txt" -type f -delete
!rm MSRDailyActivity3D/MSRDailyAct3D_pack2.zip

!unzip MSRDailyActivity3D/MSRDailyAct3D_pack3.zip -d data
!find . -name "*.bin" -type f -delete
!find . -name "*.txt" -type f -delete
!rm MSRDailyActivity3D/MSRDailyAct3D_pack3.zip

!unzip MSRDailyActivity3D/MSRDailyAct3D_pack4.zip -d data
!find . -name "*.bin" -type f -delete
!find . -name "*.txt" -type f -delete
!rm MSRDailyActivity3D/MSRDailyAct3D_pack4.zip

!unzip MSRDailyActivity3D/MSRDailyAct3D_pack5.zip -d data
!find . -name "*.bin" -type f -delete
!find . -name "*.txt" -type f -delete
!rm MSRDailyActivity3D/MSRDailyAct3D_pack5.zip

!unzip MSRDailyActivity3D/MSRDailyAct3D_pack6.zip -d data
!find . -name "*.bin" -type f -delete
!find . -name "*.txt" -type f -delete
!rm MSRDailyActivity3D/MSRDailyAct3D_pack6.zip

!unzip MSRDailyActivity3D/MSRDailyAct3D_pack7.zip -d data
!find . -name "*.bin" -type f -delete
!find . -name "*.txt" -type f -delete
!rm MSRDailyActivity3D/MSRDailyAct3D_pack7.zip

!unzip MSRDailyActivity3D/MSRDailyAct3D_pack8.zip -d data
!find . -name "*.bin" -type f -delete
!find . -name "*.txt" -type f -delete
!rm MSRDailyActivity3D/MSRDailyAct3D_pack8.zip

%cd data
!rm MSRDailyAct3DSampleCode.zip
!ls
%cd ..

Archive:  MSRDailyActivity3D/MSRDailyAct3D_pack1.zip
  inflating: data/a02_s10_e01_depth.bin  
  inflating: data/a02_s10_e01_rgb.avi  
  inflating: data/a02_s10_e01_skeleton.txt  
  inflating: data/a02_s10_e02_depth.bin  
  inflating: data/a02_s10_e02_rgb.avi  
  inflating: data/a02_s10_e02_skeleton.txt  
  inflating: data/a01_s01_e01_depth.bin  
  inflating: data/a01_s01_e01_rgb.avi  
  inflating: data/a01_s01_e01_skeleton.txt  
  inflating: data/a01_s01_e02_depth.bin  
  inflating: data/a01_s01_e02_rgb.avi  
  inflating: data/a01_s01_e02_skeleton.txt  
  inflating: data/a01_s02_e01_depth.bin  
  inflating: data/a01_s02_e01_rgb.avi  
  inflating: data/a01_s02_e01_skeleton.txt  
  inflating: data/a01_s02_e02_depth.bin  
  inflating: data/a01_s02_e02_rgb.avi  
  inflating: data/a01_s02_e02_skeleton.txt  
  inflating: data/a01_s03_e01_depth.bin  
  inflating: data/a01_s03_e01_rgb.avi  
  inflating: data/a01_s03_e01_skeleton.txt  
  inflating: data/a01_s03_e02_depth.bin  
  inflating: dat

In [ ]:
class VideoDataset(Dataset):
    def __init__(self, data_path, video_transforms=None, frame_transforms=None):
        self.files = os.listdir(data_path)
        self.data_path = data_path
        self.video_transforms = video_transforms
        self.frame_transforms = frame_transforms
        self.num_samples = len(self.files)
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        file_name = self.files[idx]
        label = int(file_name.split('_')[0][1:]) - 1
        vid = read_video(os.path.join(self.data_path, file_name), output_format='TCHW', pts_unit='sec')[0]
        
        if self.frame_transforms is not None:
            vid = torch.stack(self.frame_transforms(frame) for frame in vid)
        vid = vid.transpose(0,1) # CTHW
        if self.video_transforms is not None:
            vid = self.video_transforms(vid)
            
        return vid, torch.tensor(label)


In [ ]:
from pytorchvideo.transforms import ( # Input(C,T,H,W)
    ApplyTransformToKey,
    RandomShortSideScale,
    ShortSideScale,
    UniformTemporalSubsample
)
from torchvision.transforms import (
    Compose,
    Lambda,
)
from torchvision.transforms._transforms_video import ( # Input: (C,T,H,W)
    CenterCropVideo,
    NormalizeVideo,
    RandomCropVideo,
    RandomHorizontalFlipVideo
)


random_crop_size = 256
img_size = 224
num_frames = 16

video_train_transforms = Compose([
    UniformTemporalSubsample(num_frames),
    Lambda(lambda x: x / 255.0),
    # Lambda(lambda x: torch.transpose(x, 0, 1)), # (TCHW) -> (CTHW)
    NormalizeVideo((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
    RandomShortSideScale(min_size=random_crop_size, max_size=320),
    RandomCropVideo(img_size),
    RandomHorizontalFlipVideo(p=0.5),
])


video_val_transforms = Compose([
    UniformTemporalSubsample(num_frames),
    Lambda(lambda x: x / 255.0),
    # Lambda(lambda x: torch.transpose(x, 0, 1)), # (TCHW) -> (CTHW)
    NormalizeVideo((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
    ShortSideScale(random_crop_size),
    CenterCropVideo(img_size)
])

dataset = VideoDataset('./data', video_train_transforms)
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2], generator=generator)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

print('train len: ', len(train_dataset))
print('val len: ', len(val_dataset))

train len:  256
val len:  64


# Mini ViT

In [ ]:
sys.path.insert(0, '/content/Cream/MiniViT/Mini-DeiT/')

In [ ]:
import torch
import torch.nn as nn
from functools import partial

from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.models.helpers import load_pretrained
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from timm.models.resnet import resnet26d, resnet50d
from timm.models.registry import register_model
from timm.models.vision_transformer import _cfg, default_cfgs,\
    Mlp, PatchEmbed
try:
    from timm.models.vision_transformer import HybridEmbed
except ImportError:
    # for higher version of timm
    from timm.models.vision_transformer_hybrid import HybridEmbed

from irpe import build_rpe


class RepeatedModuleList(nn.Module):
    def __init__(self, instances, repeated_times):
        super().__init__()
        assert len(instances) == repeated_times
        self.instances = nn.ModuleList(instances)
        self.repeated_times = repeated_times
    def forward(self, *args, **kwargs):
        r = self._repeated_id
        return self.instances[r](*args, **kwargs)
    def __repr__(self):
        msg = super().__repr__()
        msg += f'(repeated_times={self.repeated_times})'
        return msg


class MiniAttention(nn.Module):
    '''
    Attention with image relative position encoding
    '''

    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., rpe_config=None, repeated_times=1, use_transform=False):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        # image relative position encoding
        rpe_qkvs = []
        for _ in range(repeated_times):
            rpe_qkv = build_rpe(rpe_config,
                      head_dim=head_dim,
                      num_heads=num_heads)
            rpe_qkvs.append(rpe_qkv)
        assert len(rpe_qkvs) == repeated_times
        assert all(len(r) == 3 for r in rpe_qkvs)
        rpe_q, rpe_k, rpe_v = zip(*rpe_qkvs)
        if rpe_q[0] is not None:
            self.rpe_q = RepeatedModuleList(rpe_q, repeated_times)
        else:
            self.rpe_q = None
        if rpe_k[0] is not None:
            self.rpe_k = RepeatedModuleList(rpe_k, repeated_times)
        else:
            self.rpe_k = None
        if rpe_v[0] is not None:
            self.rpe_v = RepeatedModuleList(rpe_v, repeated_times)
        else:
            self.rpe_v = None

        if use_transform:
            transform_bias = False
            self.conv_l = RepeatedModuleList([nn.Conv2d(num_heads, num_heads, kernel_size=1, bias=transform_bias) \
                    for _ in range(repeated_times)], repeated_times)
            self.conv_w = RepeatedModuleList([nn.Conv2d(num_heads, num_heads, kernel_size=1, bias=transform_bias) \
                    for _ in range(repeated_times)], repeated_times)
        else:
            self.conv_l = self.conv_w = None


    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)

        q *= self.scale

        attn = (q @ k.transpose(-2, -1))

        # image relative position on keys
        if self.rpe_k is not None:
            attn += self.rpe_k(q)

        # image relative position on queries
        if self.rpe_q is not None:
            attn += self.rpe_q(k * self.scale).transpose(2, 3)

        if self.conv_l is not None:
            attn = self.conv_l(attn)

        attn = attn.softmax(dim=-1)

        if self.conv_w is not None:
            attn = self.conv_w(attn)

        attn = self.attn_drop(attn)

        out = attn @ v

        # image relative position on values
        if self.rpe_v is not None:
            out += self.rpe_v(attn)


        x = out.transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

    def init_weights(self):
        def _init_weights(m):
            if isinstance(m, nn.Conv2d):
                trunc_normal_(m.weight, std=.02)
                if isinstance(m, nn.Conv2d) and m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        for m in [self.conv_l, self.conv_w]:
            if m is not None:
                m.apply(_init_weights)


class MiniBlock(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_paths=[0.], act_layer=nn.GELU, norm_layer=nn.LayerNorm, rpe_config=None, repeated_times=1, use_transform=False):
        super().__init__()
        assert len(drop_paths) == repeated_times

        self.norm1 = norm_layer(dim)
        self.norm2 = norm_layer(dim)

        self.attn = MiniAttention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop, rpe_config=rpe_config,
            repeated_times=repeated_times,
            use_transform=use_transform)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_paths = nn.ModuleList([DropPath(drop_path) if drop_path > 0. else nn.Identity() for drop_path in drop_paths])
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        drop_path = self.drop_paths[self._repeated_id]
        x = x + drop_path(self.attn(self.norm1(x)))
        x = x + drop_path(self.mlp(self.norm2(x)))
        return x


class RepeatedMiniBlock(nn.Module):
    def __init__(self, repeated_times, **kwargs):
        super().__init__()
        self.repeated_times = repeated_times
        self.block = MiniBlock(repeated_times=repeated_times, **kwargs)

        def set_repeated_times_fn(m):
            m._repeated_times = repeated_times
        self.apply(set_repeated_times_fn)

    def forward(self, x):
        for i, t in enumerate(range(self.repeated_times)):
            def set_repeated_id(m):
                m._repeated_id = i
            self.block.apply(set_repeated_id)
            x = self.block(x)
        return x

    def __repr__(self):
        msg = super().__repr__()
        msg += f'(repeated_times={self.repeated_times})'
        return msg


class VisionTransformer(nn.Module):
    """ Vision Transformer with support for patch or hybrid CNN input stage
                           and image relative position encoding
    """

    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=1000, embed_dim=768, depth=12,
                 num_heads=12, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop_rate=0., attn_drop_rate=0.,
                 drop_path_rate=0., hybrid_backbone=None, norm_layer=nn.LayerNorm, rpe_config=None,
                 use_cls_token=True,
                 repeated_times=1,
                 use_transform=False):
        super().__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models

        if hybrid_backbone is not None:
            self.patch_embed = HybridEmbed(
                hybrid_backbone, img_size=img_size, in_chans=in_chans, embed_dim=embed_dim)
        else:
            self.patch_embed = PatchEmbed(
                img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches

        if use_cls_token:
            self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        else:
            self.cls_token = None
        pos_embed_len = 1 + num_patches if use_cls_token else num_patches
        self.pos_embed = nn.Parameter(torch.zeros(1, pos_embed_len, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule

        assert depth % repeated_times == 0
        depth //= repeated_times

        blocks = []

        block_kwargs = dict(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate,
                norm_layer=norm_layer, rpe_config=rpe_config,
                use_transform=use_transform)

        for i in range(depth):
            if repeated_times > 1:
                block = RepeatedMiniBlock(
                        repeated_times=repeated_times,
                        drop_paths=dpr[i * repeated_times : (i + 1) * repeated_times],
                        **block_kwargs,
                    )
            else:
                block = MiniBlock(drop_paths=[dpr[i]], **block_kwargs)
            blocks.append(block)
        self.blocks = nn.ModuleList(blocks)

        self.norm = norm_layer(embed_dim)

        # NOTE as per official impl, we could have a pre-logits representation dense layer + tanh here
        #self.repr = nn.Linear(embed_dim, representation_size)
        #self.repr_act = nn.Tanh()

        # Classifier head
        self.head = nn.Linear(embed_dim, num_classes) if num_classes > 0 else nn.Identity()

        if not use_cls_token:
            self.avgpool = nn.AdaptiveAvgPool1d(1)
        else:
            self.avgpool = None

        trunc_normal_(self.pos_embed, std=.02)
        if self.cls_token is not None:
            trunc_normal_(self.cls_token, std=.02)
        self.apply(self._init_weights)
        self.apply(self._init_custom_weights)

        def set_repeated_id(m):
            m._repeated_id = 0
        self.apply(set_repeated_id)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def _init_custom_weights(self, m):
        if hasattr(m, 'init_weights'):
            m.init_weights()

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'pos_embed', 'cls_token'}

    def get_classifier(self):
        return self.head

    def reset_classifier(self, num_classes, global_pool=''):
        self.num_classes = num_classes
        self.head = nn.Linear(self.embed_dim, num_classes) if num_classes > 0 else nn.Identity()

    def forward_features(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)

        if self.cls_token is not None:
            cls_tokens = self.cls_token.expand(B, -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
            x = torch.cat((cls_tokens, x), dim=1)

        x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.blocks:
            x = blk(x)

        x = self.norm(x)
        if self.cls_token is not None:
            return x[:, 0]
        else:
            return x

    def forward(self, x):
        x = self.forward_features(x)
        if self.avgpool is not None:
            x = self.avgpool(x.transpose(1, 2))  # (B, C, 1)
            x = torch.flatten(x, 1)
        x = self.head(x)
        return x

In [ ]:
# tiny_vit = VisionTransformer(
#         patch_size=16, embed_dim=384, depth=12, num_heads=6, mlp_ratio=4, qkv_bias=True,
#         norm_layer=partial(nn.LayerNorm, eps=1e-6))
# tiny_vit.default_cfg = _cfg()

# checkpoint = torch.hub.load_state_dict_from_url(
#     url="https://dl.fbaipublicfiles.com/deit/deit_small_patch16_224-cd65a155.pth",
#     map_location="cpu", check_hash=True
# )
# tiny_vit.load_state_dict(checkpoint["model"])

# ViViT Model

In [ ]:
import copy

class PatchEmbedding(nn.Module):
    def __init__(self, patch_info, in_chan=3, hidden_dim=512):
        super().__init__()
        fh, fw, ft = patch_info
        self.embedding = nn.Conv3d(in_chan, hidden_dim, kernel_size=(ft,fh,fw), stride=(ft,fh,fw)) # Input of Conv3D: (N, C, T, H, W)

    def forward(self,X):
        # X: a batch of video - (N, C, T, H, W)
        x = self.embedding(X) # (N, hidden_dim, t_out, h_out, w_out)
        return x.flatten(3).permute(0, 2, 3, 1) # (N, t, #spacial_patches, hidden_dim)

class Encoder(nn.Module):
    def __init__(self, hidden_dim=192, depth=4, mlp_ratio=4, num_heads=3, spatial=False):
        super().__init__()

        tiny_vit = VisionTransformer(
            patch_size=16, embed_dim=hidden_dim, depth=12, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=True,
            norm_layer=partial(nn.LayerNorm, eps=1e-6))
        tiny_vit.default_cfg = _cfg()
        
        if spatial: # Using pretrained weight for spatial encoder
            checkpoint = torch.hub.load_state_dict_from_url(
                url="https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth",
                map_location="cpu", check_hash=True
            )
            tiny_vit.load_state_dict(checkpoint["model"])
        # Random init for temporal encoder

        self.layers = copy.deepcopy(tiny_vit.blocks[:depth])
        self.norm = nn.LayerNorm(hidden_dim)
    
    def forward(self, x):
        for blk in self.layers:
            x = blk(x)
        return self.norm(x)

In [ ]:
class ViViT(nn.Module):
    def __init__(self, img_size, patch_size, num_classes, num_frames, frame_stride, 
                 hidden_dim=192, depth=4, num_heads=3, mlp_ratio=4., emb_dropout=0.):
        # frame_stride: number of frames in a tublet embedding
        super().__init__()

        self.patch_embedding = PatchEmbedding((patch_size, patch_size, frame_stride), hidden_dim=hidden_dim)
        num_patches = (img_size // patch_size) ** 2
        temporal_dim = num_frames // frame_stride
        
        self.pos_embedding = nn.Parameter(torch.randn(1, temporal_dim, num_patches + 1, hidden_dim)) # (b, t, n+1, d)
        self.space_token = nn.Parameter(torch.randn(1, 1, 1, hidden_dim)) # (1,1,1,d)
        self.space_encoder = Encoder(hidden_dim, depth, mlp_ratio, num_heads, spatial=True)

        self.temporal_token = nn.Parameter(torch.randn(1,1, hidden_dim))
        self.temporal_encode = Encoder(hidden_dim, depth, mlp_ratio, num_heads)

        self.dropout = nn.Dropout(emb_dropout)

        self.head = nn.Linear(hidden_dim, num_classes)

        self.patch_embedding.apply(self._init_embedding)

    def _init_embedding(self, m):
        """Initialisation function for 3D convolutional kernels.

        The filter is initialised such that it only depends on the input at the
        central (w.r.t the time dimension) frame.
        """

        if isinstance(m, nn.Conv3d):
            torch.nn.init.normal_(m.weight)
            central_frame_idx = m.weight.shape[2] // 2
            m.weight.data[:, :, :central_frame_idx, :, :].fill_(0.0)
            m.weight.data[:, :, (central_frame_idx+1):, :, :].fill_(0.0)


    def forward(self, x):
        x = self.patch_embedding(x)
        b, t, n, d = x.shape
        
        # Space encode
        x = torch.cat((self.space_token.expand(b, t, -1, -1), x), dim=2) # space_token expand: (b,t,1,d) -> x: (b,t,n+1,d)
        x += self.pos_embedding
        x = self.dropout(x)

        x = x.view(-1, n+1, d) # (b*t, n+1, d)
        x = self.space_encoder(x) # (b*t, n+1, d)

        # Temporal encode
        x = x[:, 0].view(b, t, d) # Get t space_cls tokens of 1 example -> b*t tokens of b examples
        x = torch.cat((self.temporal_token.expand(b, -1, -1), x), dim=1) # temporal_token expand: (b, 1, d) -> x: (b,t+1,d)
        x = self.temporal_encode(x) # (b, t+1, d)

        return self.head(x[:, 0])


In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(model):
    model.eval()
    total, acc_count = 0, 0
    total_loss = 0
    with torch.no_grad():
        for step, (vid, label) in enumerate(val_dataloader):
            vid, label = vid.to(device), label.to(device)
            logit = model(vid)
            loss = criterion(logit, label)
            total_loss += loss.item()
            
            prob = F.softmax(logit, dim=1)
            pred = torch.argmax(prob, dim=1)
            total += vid.shape[0]
            acc_count += accuracy_score(label.cpu(), pred.cpu(), normalize=False)
    
    return acc_count/total, total_loss/len(val_dataloader)


# Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
img_size = 224
patch_size = 16
num_classes = 16
frame_stride = 2


model = ViViT(img_size, patch_size, num_classes, num_frames, frame_stride, emb_dropout=0.5)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.1)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 100)
criterion = nn.CrossEntropyLoss()

In [ ]:
epochs = 20
best_acc = 0

model.train()
for epoch in range(epochs):
    tbar = tqdm(train_dataloader)
    total_loss = 0
    start_time = time.time()
    for step, (vid, label) in enumerate(tbar):
        vid, label = vid.to(device), label.to(device)
        logit = model(vid)
        loss = criterion(logit, label)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tbar.set_description("Epoch: {}/{} - Learning rate: {:.6f} - train Loss: {:.6f}".format(epoch+1, 
                                                                                            epochs,
                                                                                            optimizer.state_dict()['param_groups'][0]['lr'], 
                                                                                            loss.item()))

    scheduler.step()
    end_time = time.time()

    # EVAL
    acc, val_loss = evaluate(model)
    print(f"train loss: {total_loss/len(train_dataloader)} | val loss: {val_loss} | val accuracy: {acc*100} | time: {end_time-start_time}")

    # Save checkpoint
    if acc > best_acc:
        best_acc = acc
        ckpt = {'model':model.state_dict(),
                'best_acc':best_acc}
        torch.save(ckpt, 'checkpoint.pt')
        print('checkpoint is saved!')
        

Epoch: 1/20 - Learning rate: 0.001000 - train Loss: 2.854367: 100%|██████████| 16/16 [03:13<00:00, 12.07s/it]


train loss: 2.9636755138635635 | val loss: 2.970923900604248 | val accuracy: 3.125 | time: 193.0877583026886
checkpoint is saved!


Epoch: 2/20 - Learning rate: 0.001000 - train Loss: 2.895894: 100%|██████████| 16/16 [03:15<00:00, 12.24s/it]


train loss: 2.859984964132309 | val loss: 2.896196722984314 | val accuracy: 4.6875 | time: 195.8059046268463
checkpoint is saved!


Epoch: 3/20 - Learning rate: 0.000999 - train Loss: 2.714729: 100%|██████████| 16/16 [03:15<00:00, 12.20s/it]


...

train loss: 2.785442665219307 | val loss: 2.84832763671875 | val accuracy: 3.125 | time: 185.19383025169373


Epoch: 14/20 - Learning rate: 0.000959 - train Loss: 2.776619: 100%|██████████| 16/16 [03:12<00:00, 12.00s/it]


train loss: 2.7825259417295456 | val loss: 2.8354455828666687 | val accuracy: 3.125 | time: 192.0168812274933


Epoch: 15/20 - Learning rate: 0.000952 - train Loss: 2.771802:   6%|▋         | 1/16 [00:16<04:13, 16.88s/it]


KeyboardInterrupt: ignored

In [ ]:
# ! fuser -v /dev/nvidia*

In [ ]:
# !kill -9 383